# Lab Three: Extending Logistic Regression
## Caleb Moore, Blake Gebhardt, Christian Gould
dataset: https://www.kaggle.com/datasets/open-powerlifting/powerlifting-database

In [22]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import datasets

In [23]:
# Notebook setup
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

# Preparation and Overview (3 points total)
* [2 points] Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results. For example, would the model be deployed or used mostly for offline analysis? As in previous labs, also detail how good the classifier needs to perform in order to be useful.

# Background
### Where did our data come from?
* Our data came from the Open Powerlifting Databse. They are trying to create a database of powerlifting competitions for the public domain. The dataset that we are using here is a snapshot of what the database looked like as of April 2019. Powerlifting is a sport in which competitors compete to lift the most weight for their class in three separate barbell lifts: the Squat, Bench, and Deadlift. The dataset that we have here contains information about the competitors, the competitions, and the lifts that they performed.

### What could we do with this data?
* There is a lot of data that we can use here, but what we are more focused on is the prediction of the weight class of each person. If we can use data to predict with a very high accuracy what weight class a person should be in, it could be helpful in determining if someone is cheating or not. People could lie about their weight if there is not a weighing time. This could also show if someone is likely to be taking steroids. If they are classified as being far above the weight class than what they actually are, then investigators could look into the possiblity of them taking steroids.

### What is the task we are trying to solve?
* Given all the information that we have, specifically:
  * Sex
    * Sex has a large impact on how much the lifter can lift
  * Equipment
    * If the lifter uses straps or other apparatus, it can affect how much they can lift
  * Age
    * Age has a large impact on how much the lifter can lift
  * Division
    * A higher skill division can mean that the lifter can lift more
  * BodyweightKg
    * The actual weight of the lifter is a large factor in how much they can lift
  * Best3SquatKg
    * The best squat that the lifter did in 3 attempts
  * Best3BenchKg
    * The best bench that the lifter did in 3 attempts
  * Best3DeadliftKg
    * The best deadlift that the lifter did in 3 attempts
  * Tested
    * Whether or not the lifter was tested for drugs
  * Federation
    * The federation that held the competition

We want to be able to reliably predict what Weight Class that person should be in. The target column for that data is WeightClassKg.

The rest of the columns are not as important to us, so we will be dropping them.

The columns to be dropped are:
* Name
  * The name of the lifter is irrelevant to the weight class that they should be in
* Event
  * The federation that held the event is irrelevant to the weight class that they should be in
* TotalKg
  * The total weight lifted by the lifter. This is redundant since it is just the sum of the best squat, best bench, and best deadlift, which we already have
* AgeClass
  * Because we already have the age of the lifter, this column is redundant
* Squat1Kg - Squat4Kg
  * These are the individual squat attempts, we only really care about their best one
* Bench1Kg - Bench4Kg
  * These are the individual bench attempts, we only really care about their best one
* Deadlift1Kg - Deadlift4Kg
  * These are the individual deadlift attempts, we only really care about their best one
* Place
  * We don't need to know where the competition was held, just the results
* Wilks
  * This is a calculation of the total weight lifted in relation to the weight of the lifter, which is information we are already accounting for, so it would be redundant
* McCulloch
  * This is another calculation method like Wilks, and for the same reason as Wilks, we will not need this.
* Glossbrenner
  * This is another calculation method like Wilks, and for the same reason as Wilks, we will not need this.
* IPFPoints
  * This is a calculation of the total weight lifted in relation to the weight of the lifter, which is information we are already accounting for, so it would be redundant
* MeetCountry
  * We don't need to know where the competition was held, just the results
* MeetState
  * We don't need to know where the competition was held, just the results
* MeetName
  * We don't need to know the name of the meet, just what was lifted
* Country
  * We don't need to know where the lifter is from, just what they lifted
* Date
  * We don't need to know when the competition was held, just what was lifted

### What are we aiming for

* If we want our classifier to be reliable, we need to have at least 90% accuracy. If we can get to that, then we can be confident that we can use this classifier to determine if someone is possibly cheating or taking steroids. That level of accuracy would be very useful for investigators. 


In [24]:
cols_to_drop = [
    'Name',
    'Event',
    'TotalKg',
    'AgeClass',
    'Squat1Kg',
    'Squat2Kg',
    'Squat3Kg',
    'Squat4Kg',
    'Bench1Kg',
    'Bench2Kg',
    'Bench3Kg',
    'Bench4Kg',
    'Deadlift1Kg',
    'Deadlift2Kg',
    'Deadlift3Kg',
    'Deadlift4Kg',
    'Place',
    'Wilks',
    'McCulloch',
    'Glossbrenner',
    'IPFPoints',
    'MeetCountry',
    'MeetState',
    'MeetName',
    'Country',
    'Date',
]

[.5 points] (mostly the same processes as from previous labs) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created). Provide a breakdown of the variables after preprocessing (such as the mean, std, etc. for all variables, including numeric and categorical).

# Initial Data Exploration
* We will start by importing the data and looking at the first few rows to get a feel for what we are working with.

* Some of the rows have NaN values, specifically in the 3 main lifts. We will give those NaN values a value of 0, since those people did not lift anything. Some of the meets did not have competitors do all 3 lifts, and we want to keep those rows, so we will just give them a value of 0.

* We will also drop the rows that have NaN values in the WeightClassKg column, since we want to predict that column.

* We took a look at how many weight classes there were, and there ended up being 13. A couple of the weight classes were in a different format, where it was 140+ or 110+ instead of the typical weight class, and we will leave those unaffected since some federations use that format.

* We also found that the 'Tested' Column held nothing by NaN values, so we will drop that column as well.

In [25]:
# lets look at the data
df = pd.read_csv('data/openpowerlifting.csv', nrows=100)
# Drop the columns we don't need
df = df.drop(cols_to_drop, axis=1)
print(df.shape)
df.head()

(100, 11)


,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,Tested,Federation
0,F,Wraps,29,F-OR,59.8,60,105.0,55.0,130.0,NaN,GPC-AUS
1,F,Wraps,29,F-OR,58.5,60,120.0,67.5,145.0,NaN,GPC-AUS
2,F,Raw,40,F-OR,55.4,56,NaN,32.5,NaN,NaN,GPC-AUS
3,F,Wraps,23,F-OR,60.0,60,105.0,72.5,132.5,NaN,GPC-AUS
4,F,Wraps,45,F-OR,104.0,110,140.0,80.0,170.0,NaN,GPC-AUS


### Look at the types of Weight Classes

In [26]:
# Lets look at some unique values that might be worth encoding
print(df['WeightClassKg'].unique())
print(len(df['WeightClassKg'].unique()))

['60' '56' '110' '75' '82.5' '52' '67.5' '90' '110+' '125' '100' '140'
 '140+']
13


### Tested is NaN only
* So lets drop that column

In [27]:
print(df['Tested'].unique())
df = df.drop('Tested', axis=1)

df.head()

[nan]


,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,Federation
0,F,Wraps,29,F-OR,59.8,60,105.0,55.0,130.0,GPC-AUS
1,F,Wraps,29,F-OR,58.5,60,120.0,67.5,145.0,GPC-AUS
2,F,Raw,40,F-OR,55.4,56,NaN,32.5,NaN,GPC-AUS
3,F,Wraps,23,F-OR,60.0,60,105.0,72.5,132.5,GPC-AUS
4,F,Wraps,45,F-OR,104.0,110,140.0,80.0,170.0,GPC-AUS


### Look at the types of Divisions
  * Clearly, we should encode this as a one-hot encoding since there are only 3 possible values

In [28]:
print(df['Division'].unique())

# One hot encode that column
df = pd.get_dummies(df, columns=['Division'])
df.head()

['F-OR' 'M-OR' 'M-OE']


,Sex,Equipment,Age,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,Federation,Division_F-OR,Division_M-OE,Division_M-OR
0,F,Wraps,29,59.8,60,105.0,55.0,130.0,GPC-AUS,1,0,0
1,F,Wraps,29,58.5,60,120.0,67.5,145.0,GPC-AUS,1,0,0
2,F,Raw,40,55.4,56,NaN,32.5,NaN,GPC-AUS,1,0,0
3,F,Wraps,23,60.0,60,105.0,72.5,132.5,GPC-AUS,1,0,0
4,F,Wraps,45,104.0,110,140.0,80.0,170.0,GPC-AUS,1,0,0


### Federation
* There is only one possible federation, so we will actually drop that column

In [31]:
print(df['Federation'].unique())

# Drop the column
df = df.drop('Federation', axis=1)
df.head()

['GPC-AUS']


,Sex,Equipment,Age,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,Division_F-OR,Division_M-OE,Division_M-OR
0,F,Wraps,29,59.8,60,105.0,55.0,130.0,1,0,0
1,F,Wraps,29,58.5,60,120.0,67.5,145.0,1,0,0
2,F,Raw,40,55.4,56,NaN,32.5,NaN,1,0,0
3,F,Wraps,23,60.0,60,105.0,72.5,132.5,1,0,0
4,F,Wraps,45,104.0,110,140.0,80.0,170.0,1,0,0


### Look at the Equipment
* Since there are only 3 options, we can one-hot encode that column

In [33]:
print(df['Equipment'].unique())

# One hot encode that column
df = pd.get_dummies(df, columns=['Equipment'])
df.head()

['Wraps' 'Raw' 'Single-ply']


,Sex,Age,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,Division_F-OR,Division_M-OE,Division_M-OR,Equipment_Raw,Equipment_Single-ply,Equipment_Wraps
0,F,29,59.8,60,105.0,55.0,130.0,1,0,0,0,0,1
1,F,29,58.5,60,120.0,67.5,145.0,1,0,0,0,0,1
2,F,40,55.4,56,NaN,32.5,NaN,1,0,0,1,0,0
3,F,23,60.0,60,105.0,72.5,132.5,1,0,0,0,0,1
4,F,45,104.0,110,140.0,80.0,170.0,1,0,0,0,0,1


### Sex
* Since there are only two options here, we will one-hot encode this column as well

In [35]:
print(df['Sex'].unique())

# One hot encode that column
df = pd.get_dummies(df, columns=['Sex'])
df.head()

['F' 'M']


,Age,BodyweightKg,WeightClassKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,Division_F-OR,Division_M-OE,Division_M-OR,Equipment_Raw,Equipment_Single-ply,Equipment_Wraps,Sex_F,Sex_M
0,29,59.8,60,105.0,55.0,130.0,1,0,0,0,0,1,1,0
1,29,58.5,60,120.0,67.5,145.0,1,0,0,0,0,1,1,0
2,40,55.4,56,NaN,32.5,NaN,1,0,0,1,0,0,1,0
3,23,60.0,60,105.0,72.5,132.5,1,0,0,0,0,1,1,0
4,45,104.0,110,140.0,80.0,170.0,1,0,0,0,0,1,1,0


In [ ]:
print(df['Type of Admission'].unique())

['Emergency' 'Trauma' 'Urgent']


In [ ]:
print(df['Severity of Illness'].unique())

['Extreme' 'Moderate' 'Minor']


In [ ]:
print(df['Age'].unique())
print(df['Age'].value_counts())

['51-60' '71-80' '31-40' '41-50' '81-90' '61-70' '21-30']
31-40    31
71-80    19
51-60    18
81-90    13
41-50     7
61-70     7
21-30     5
Name: Age, dtype: int64


In [ ]:
# lets get rid of any na values since we have enough raw data 
total_with_na = df.shape[0]
df.dropna(inplace=True)
print('dropped', total_with_na - df.shape[0], 'values')

dropped 0 values


In [ ]:
# separate the target column
train_stays = df['Stay']
df.drop('Stay', axis=1, inplace=True)

In [ ]:
# lets make a map to replace string values
mapping = {     '0-10': 1,
                '11-20': 2,
                '21-30': 3,
                '31-40': 4,
                '41-50': 5,
                '51-60': 6,
                '61-70': 7,
                '71-80': 8,
                '81-90': 9,
                '91-100': 10,

                'Minor': 0,
                'Moderate': 1,
                'Extreme': 2,
                
                'Urgent': 0,
                'Trauma': 1,
                'Emergency': 2,

                'A': 0,
                'B': 1,
                'C': 2,
                'D': 3,
                'E': 4,
                'F': 5,

                'P': 0,
                'Q': 1,
                'R': 2,
                'S': 3,
                'T': 4,

                'X': 0,
                'Y': 1,
                'Z': 2,

                'radiotherapy': 0,
                'anesthesia': 1,
                'gynecology': 2,
                'TB & Chest disease': 3,
                'surgery': 4,

                'a': 0,
                'b': 1,
                'c': 2,
                'd': 3,
                'e': 4,
                'f': 5,
                'g': 6
        }

In [ ]:
df = df.replace(mapping)
df

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,1,8,2,3,2,3,0,2,5,2.0,31397,7.0,2,2,2,6,4911.0
1,2,2,2,5,2,2,0,3,5,2.0,31397,7.0,1,2,2,6,5954.0
2,3,10,4,1,0,2,1,3,4,2.0,31397,7.0,1,2,2,6,4745.0
3,4,26,1,2,1,2,0,2,3,2.0,31397,7.0,1,2,2,6,7272.0
4,5,26,1,2,1,2,0,3,3,2.0,31397,7.0,1,2,2,6,5558.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1,3,10,1,4,2,1,1,1.0,21257,7.0,0,1,2,4,3750.0
96,97,13,0,5,2,3,2,2,5,2.0,21257,7.0,0,1,2,4,6162.0
97,98,15,2,5,2,4,2,2,5,2.0,21257,7.0,0,1,3,4,5199.0
98,99,12,0,9,1,3,2,2,1,1.0,21257,7.0,0,1,2,4,4796.0


In [ ]:
# Lets take a look at it now
print(df.shape)
df.head()

(100, 17)


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,1,8,2,3,2,3,0,2,5,2.0,31397,7.0,2,2,2,6,4911.0
1,2,2,2,5,2,2,0,3,5,2.0,31397,7.0,1,2,2,6,5954.0
2,3,10,4,1,0,2,1,3,4,2.0,31397,7.0,1,2,2,6,4745.0
3,4,26,1,2,1,2,0,2,3,2.0,31397,7.0,1,2,2,6,7272.0
4,5,26,1,2,1,2,0,3,3,2.0,31397,7.0,1,2,2,6,5558.0


[.5 points] Divide your data into training and testing data using an 80% training and 20% testing split. Use the cross validation modules that are part of scikit-learn. Argue "for" or "against" splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset?

* Using an 80/20 split makes sense, as we have a large dataset with a lot of features. We want to make sure that we have enough data to train our model, but we also want to make sure that we have enough data to test it as well. We don't want to overfit our model as a result of using too little data, so we will stick with the 80/20 split.

* *the data was already split for us, so we will practice splitting and validating with the train data since it's adequately large*

In [ ]:
print('original shapes')
print('X:', df.shape)
print('y:', train_stays.shape)
print()

X_train, X_test, y_train, y_test = train_test_split(df, train_stays, test_size=0.2, random_state=0)

print('train shapes')
print('X:', X_train.shape)
print('y', y_train.shape)
print()

print('test shapes')
print('X:', X_test.shape)
print('y:', y_test.shape)
# clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
# clf.score(X_test, y_test)

original shapes
X: (100, 17)
y: (100,)

train shapes
X: (80, 17)
y (80,)

test shapes
X: (20, 17)
y: (20,)


In [ ]:
X, y = datasets.load_iris(return_X_y=True)
print(X)
y

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

# Modeling (5 points total)
The implementation of logistic regression must be written only from the examples given to you by the instructor. No credit will be assigned to teams that copy implementations from another source, regardless of if the code is properly cited. 

[2 points] Create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. Use object oriented conventions identical to scikit-learn. You should start with the template developed by the instructor in the course. You should add the following functionality to the logistic regression classifier:

* Ability to choose optimization technique when class is instantiated: either steepest ascent, stochastic gradient ascent, and {Newton's method/Quasi Newton methods}. 
* Update the gradient calculation to include a customizable regularization term (either using no regularization, L1 regularization, L2 regularization, or both L1 and L2 regularization). Associate a cost with the regularization term, "C", that can be adjusted when the class is instantiated.

In [ ]:
"""
"""
class LRC:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        # Number of iterations is initially 1000
        self.n_iters = n_iters
        # The weights will be made later as well as the bias, whenever we fit
        self.weights = None
        self.bias = None

    """
    Fit function.
    X: The feature matrix
    y: The target vector
    """
    def fit(self, X, y):
        # Number of samples and features
        n_samples, n_features = X.shape

        # set the beginning to have zeros for all the weights
        self.weights = np.zeros(n_features)
        # set the bias to 0 initially
        self.bias = 0

        # gradient descent
        # Used an _ since we don't need to use the iterator, we just need this number of iterations
        for _ in range(self.n_iters):
          # First thing first, calculate the dot product of the weights and the features
            linear_model = np.dot(X, self.weights) + self.bias
          # Then we pass it through the sigmoid function
            y_predicted = self.sigmoid(linear_model)

          # Then we calculate the derivatives
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

          # Then we update the weights and the bias accordingly
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    # Function to predict the values
    def predict(self, X):
      # Again, like gradient descent, we calculate the dot product and add the bias
        linear_model = np.dot(X, self.weights) + self.bias
      # Then we pass it through the sigmoid function
        y_predicted = self.sigmoid(linear_model)
      # Then we convert the values to 0 or 1
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return y_predicted_cls

    # Basic sigmoid function found online:
    # https://en.wikipedia.org/wiki/Sigmoid_function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

[1.5 points] Train your classifier to achieve good generalization performance. That is, adjust the optimization technique and the value of the regularization term(s) "C" to achieve the best performance on your test set. Visualize the performance of the classifier versus the parameters you investigated. Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

[1.5 points] Compare the performance of your "best" logistic regression optimization procedure to the procedure used in scikit-learn. Visualize the performance differences in terms of training time and classification performance. Discuss the results. 

# Deployment (1 points total)
* Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

# Exceptional Work (1 points total)
* You have free reign to provide additional analyses. One idea: Update the code to use either "one-versus-all" or "one-versus-one" extensions of binary to multi-class classification. 